<a href="https://colab.research.google.com/github/manishaachary13/Space-Launch-Mission/blob/main/mission%20launch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✈️ Project Title:  **Big Data Analytics of Global Space Launch Missions**

# **🧩 Problem Statement:**

With the rise of both government and private space agencies, the number of satellite and exploratory missions has increased rapidly. However, delays, failures, and cost overruns still pose a serious challenge. This project aims to use Big Data tools (specifically PySpark) to analyze historical space launch data, uncover patterns in successful/failed missions, and predict mission outcomes.

This project aims to utilize **Big Data analytics using PySpark** to process and analyze historical global space launch mission data. The goals are to:
* Identify patterns and trends in space launch activity over time.
* Analyze mission success/failure based on organization, rocket status, launch site, or launch type.
* Explore cost drivers in space missions and the relationship between cost and success.
* Predict mission outcomes (success/failure) using classification models.

## 🛰️ Key Features in the Dataset:


Organisation: Agency or private company (e.g., SpaceX, ISRO, NASA, Roscosmos)

Location: Launch site

Date: Mission launch date

Detail: Description of the mission

Rocket_Status: Operational status of the rocket

Price: Launch cost (in million USD)

Mission_Status: Outcome of the mission (e.g., Success, Failure, Partial Failure)

#### 📦 1. Install and Configure PySpark

In [1]:
# 1. Install Java and PySpark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install -q pyspark findspark

# 2. Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# 3. Initialize Findspark
import findspark
findspark.init()

# 4. Start Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SpaceLaunchBigData") \
    .getOrCreate()

print("✅ Spark Session Created Successfully")



✅ Spark Session Created Successfully


In [2]:
# Replace the link below with your actual raw GitHub link
url = "https://raw.githubusercontent.com/manishaachary13/Space-Launch-Mission/refs/heads/main/mission_launches.csv"

# Read CSV directly from GitHub
df = spark.read.csv(url, header=True, inferSchema=True)
df.printSchema()
df.show(5)

UnsupportedOperationException: None